# 🎚️ Лабораторная работа №4: Проектирование цифровых фильтров
**Курс:** Основы теории сигналов и систем (ОТСиС)  
**Цель:** Научиться проектировать FIR и IIR фильтры для очистки сигналов.

## 🎯 Типы фильтров
1. **Low-Pass (ФНЧ):** Пропускает низкие частоты, давит высокие.
2. **High-Pass (ФВЧ):** Пропускает высокие, давит низкие.
3. **Band-Pass (ПФ):** Пропускает полосу частот.
4. **Band-Stop (РФ):** Вырезает конкретную частоту (например, 50 Гц).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
print('✅ Библиотеки готовы!')

## 1. FIR vs IIR Фильтры

| Характеристика | FIR (Finite Impulse Response) | IIR (Infinite Impulse Response) |
|---|---|---|
| **Устойчивость** | Всегда устойчив | Может быть неустойчив |
| **Фаза** | Линейная (не искажает форму) | Нелинейная |
| **Порядок** | Высокий (требует много вычислений) | Низкий (эффективен) |
| **Пример** | `signal.firwin` | Butterworth, Chebyshev |

In [ ]:
# Задача: Убрать высокочастотный шум из ЭКГ-подобного сигнала
fs = 1000
t = np.arange(0, 2, 1/fs)

# Имитация сигнала: медленная волна (2 Гц) + шум (50 Гц)
x = np.sin(2*np.pi*2*t) + 0.5*np.sin(2*np.pi*50*t)

# Проектируем фильтр Баттерворта (Low-Pass)
# Частота среза 10 Гц (все что выше - давим)
cutoff = 10  # Гц
order = 4    # Порядок фильтра (крутизна спада)

# Создаем коэффициенты фильтра (b, a)
b, a = signal.butter(order, cutoff, btype='low', fs=fs)

# Применяем фильтр (filtfilt - фильтрация без сдвига фазы)
y = signal.filtfilt(b, a, x)

# Визуализация
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

ax1.plot(t[:200], x[:200], 'r', alpha=0.5, label='Сигнал + Шум (50 Гц)')
ax1.plot(t[:200], y[:200], 'b', linewidth=2, label='Отфильтрованный (Butterworth)')
ax1.set_title('Фильтрация во временной области')
ax1.legend()
ax1.grid(True)

# Частотная характеристика фильтра (АЧХ)
w, h = signal.freqz(b, a, fs=fs)
ax2.plot(w, 20 * np.log10(abs(h)), 'g')
ax2.set_title('АЧХ фильтра (Bode Plot)')
ax2.set_xlabel('Частота, Гц')
ax2.set_ylabel('Амплитуда, дБ')
ax2.set_xlim([0, 100])
ax2.set_ylim([-60, 5])
ax2.axvline(cutoff, color='k', linestyle='--', label='Срез 10 Гц')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Проектируем FIR фильтр методом окон
# Тот же Low-Pass 10 Гц, но FIR

numtaps = 101  # Количество коэффициентов (должно быть нечетным)
b_fir = signal.firwin(numtaps, cutoff, fs=fs)

# Фильтрация (простая свёртка)
y_fir = signal.lfilter(b_fir, 1.0, x)

# Сравнение задержек
plt.figure(figsize=(12, 4))
plt.plot(t[:100], x[:100], 'gray', alpha=0.3, label='Вход')
plt.plot(t[:100], y[:100], 'b', label='IIR (filtfilt - нет задержки)')
plt.plot(t[:100], y_fir[:100], 'g--', label='FIR (lfilter - есть задержка!)')
plt.title('Сравнение задержек фильтров')
plt.legend()
plt.grid(True)
plt.show()

print("Обратите внимание: FIR фильтр сдвигает сигнал вправо (вносит задержку).")

## 🛠 ЗАДАНИЕ: Режекторный фильтр (Notch Filter)

**Ситуация:** Вы записали звук, но в него попал гул от электросети **50 Гц**.
**Задача:** Вырезать ТОЛЬКО 50 Гц, сохранив остальное.

1. Сгенерируйте сигнал: $x(t) = \sin(2\pi \cdot 10 \cdot t) + 0.6 \sin(2\pi \cdot 50 \cdot t)$
2. Спроектируйте **Band-Stop (Notch)** фильтр Баттерворта.
   - Полоса подавления: 48–52 Гц.
3. Отфильтруйте сигнал.
4. Постройте спектры ДО и ПОСЛЕ фильтрации.

In [ ]:
# 1. Генерация
fs = 1000
t = np.arange(0, 1, 1/fs)
x_bad = np.sin(2*np.pi*10*t) + 0.6*np.sin(2*np.pi*50*t)

# 2. Проектирование Notch фильтра (Band-Stop)
# iirnotch - специальная функция для узких фильтров
f0 = 50.0  # Частота для удаления
Q = 30.0   # Качество (чем больше, тем уже вырез)
b_notch, a_notch = signal.iirnotch(f0, Q, fs)

# 3. Фильтрация
y_clean = signal.filtfilt(b_notch, a_notch, x_bad)

# 4. Спектральный анализ
def plot_spectrum(sig, name, ax, color):
    X = np.fft.fft(sig)
    freqs = np.fft.fftfreq(len(sig), 1/fs)
    ax.plot(freqs[:len(sig)//2], np.abs(X)[:len(sig)//2], color=color)
    ax.set_title(name)
    ax.grid(True)
    ax.set_xlim([0, 100])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
plot_spectrum(x_bad, 'Спектр ДО (виден пик 50 Гц)', ax1, 'r')
plot_spectrum(y_clean, 'Спектр ПОСЛЕ (пик 50 Гц удален)', ax2, 'g')
plt.show()